In [ ]:
import numpy as np
import math
import cmath

from qiskit.quantum_info import Pauli

from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.circuit.library import UnitaryGate, QFT
from scipy.linalg import expm
pi = np.pi


def create_hamiltonian(qubits, g, quiet=False):
    H = np.zeros((2**qubits, 2**qubits), dtype=np.complex128)

    # construct the Hamiltonian
    # with Pauli Operators in Qiskit ^ represents a tensor product
    if not quiet: print("H = ", end='')
    for i in range(qubits-1):
        temp = Pauli('')
        for j in range(qubits):
            if (j == i or j == i+1):
                temp ^= Pauli('Z')
            else:
                temp ^= Pauli('I')
        if not quiet: print("-"+str(temp)+" ", end='')
        H += -temp.to_matrix()
    for i in range(qubits):
        temp = Pauli('')
        for j in range(qubits):
            if (j == i):
                temp ^= Pauli('X')
            else:
                temp ^= Pauli('I')
        if not quiet: print("-"+str(g)+"*"+str(temp)+" ", end='')
        H += -g*temp.to_matrix()
    if not quiet: print("\n")

    # normalize the Hamiltonian (not needed here)
    H = (1/8)*H/np.linalg.norm(H, ord=2)
    
    # rotate matrix so that it will be positive definite
    H += 1/2*np.eye(2**qubits)

    if not quiet: print("Eigenvalues of the Hamiltonian:\n", np.linalg.eigvals(H))
    min_eigenvalue = np.min(np.linalg.eigvals(H))
    if not quiet: print("\nLowest energy Eigenvalue", min_eigenvalue)

    return H
# def adjust_ham(Ham):
#     H = Ham
#     
    
    
#     # # convert revolutions to radians
#     # H *= 2*np.pi*1j
    
#     # # convert the rotations to complex numbers
#     # H = expm(H)
#     return H
# def readjust_eigenvalues(eigenvalues):
#     E = eigenvalues
#     E -= 1/2*np.eye(2**qubits)
#     E *= (1/8)
#     return 

def run_hadamard_test(ham, t):
    # # just for testing
    # Ham  = np.array([[1,0], [0, np.exp(1j*pi)]])
    qr_ancilla = QuantumRegister(1)
    qr_eigenstate = QuantumRegister(np.log2(Ham[0].shape[0]))
    cr = ClassicalRegister(1)
    qc = QuantumCircuit(qr_ancilla, qr_eigenstate, cr)
    qc.h(qr_ancilla)
    qc.x(qr_eigenstate)
    mat = expm(1j * Ham*t)
    controlled_U = UnitaryGate(mat).control(annotated="yes")
    qc.append(controlled_U, qargs = [qr_ancilla[:]] + qr_eigenstate[:] )
    qc.h(qr_ancilla)
    qc.measure(qr_ancilla[0],cr[0])
    # print(qc)
    aer_sim = AerSimulator()
    trans_qc = transpile(qc, aer_sim)
    Nsample = 10000
    counts = aer_sim.run(trans_qc, shots = Nsample).result().get_counts()
    
    re_p0 = 0
    if counts.get('0') is not None:
        re_p0 = counts['0']/Nsample
    
    Re = 2*re_p0-1

    return Re

def generate_s_k(ham, k, Dt, K):
    # use real hadmard test to generate data
    s_k = [0 for _ in range(2*K+1)]
    for i in range(2*K+1):
        s_k[i] = run_hadamard_test(ham, i*Dt)
    return s_k

def make_X(start, K, d, s_k = []):
    X = np.zeros((d, K+1))
    for i in range(len(X)):
        for j in range(len(X[i])):
            X[i][j] = s_k[i+j+start]
    return X

def regression(X, noise_threshold):
    # print("X", X)
    U, sigma, V = np.linalg.svd(X)
    threshold = noise_threshold*max(sigma)
    # print("U", U)
    # print("sigma", sigma)
    # print("V", V)
    for l in range(len(sigma)):
        # print("iteration")
        # print("sigma_l", sigma[l])
        if sigma[l] <= threshold:
            # print("    too small\n")
            sigma[l] = 0
            U[:,l] = 0
            V[:,l] = 0
    sigma_mat = np.zeros((U.shape[1], V.shape[0]))
    for l in range(len(sigma)):
        sigma_mat[l][l] = sigma[l]
    # print("U\n", U)
    # print("sigma\n", sigma_mat)
    # print("V\n", V.conj().T)
    return U @ sigma_mat @ V.conj().T

'''
Dt: time step for each iteration (Dt should not be small)
noise_threshold: used for linear regression
err_threshold: how close the energy until iterations have been stopped
K: number of time steps to take each
alpha: used for calculating d  
'''
def ODMD(ham, Dt, noise_threshold, err_threshold, K = 3, alpha = 1/2): 
    k = 0
    # this is the suggested formula
    # d = math.floor(alpha*(K+1))
    d=3
    while (True):
        print(k)
        s_k = generate_s_k(ham, k, Dt, K)
        X = make_X(0, K, d, s_k=s_k)
        print(X)
        Xprime = make_X(1, K, d, s_k=s_k)
        print("Xprime\n",Xprime) 
        X_sigma = regression(X, noise_threshold)
        print("X_sigma\n",X_sigma)  
        A = Xprime @ np.linalg.pinv(X_sigma)
        print("A\n",A)  
        k = k+1
        eigenvalues,_ = np.linalg.eig(A)
        # I don't know if I am doing this right (I log?)
        eigenlog = [math.atan2(i.imag, i.real) for i in eigenvalues]
        print(eigenlog)
        E_0 = -max(eigenlog, key=abs)/Dt
        print("E_0 =", E_0*Dt)
        try:
            if abs(old_E_0 - E_0) < err_threshold:
                break
        except:
            True
        finally:
            old_E_0 = E_0
    return E_0

Ham = create_hamiltonian(2, 0)
print(Ham)

ODMD(Ham, .001, .000001, 0.00001)

H = -ZZ -0*XI -0*IX 

Eigenvalues of the Hamiltonian:
 [0.375+0.j 0.625+0.j 0.625+0.j 0.375+0.j]

Lowest energy Eigenvalue (0.375+0j)
[[0.375+0.j 0.   +0.j 0.   +0.j 0.   +0.j]
 [0.   +0.j 0.625+0.j 0.   +0.j 0.   +0.j]
 [0.   +0.j 0.   +0.j 0.625+0.j 0.   +0.j]
 [0.   +0.j 0.   +0.j 0.   +0.j 0.375+0.j]]
0
[[ 1.      0.8294  0.386  -0.1756]
 [ 0.8294  0.386  -0.1756 -0.7004]
 [ 0.386  -0.1756 -0.7004 -0.979 ]]
Xprime
 [[ 0.8294  0.386  -0.1756 -0.7004]
 [ 0.386  -0.1756 -0.7004 -0.979 ]
 [-0.1756 -0.7004 -0.979  -0.9208]]
X_sigma
 [[ 1.09955451  0.69757704  0.39482661  0.12733346]
 [ 0.82198227  0.24132542  0.08239118  0.7858847 ]
 [ 0.26839036 -0.27798968 -0.27869864  1.18399659]]
A
 [[ 10.86872032 -17.15783375   9.8216513 ]
 [ 14.19051122 -23.69590953  13.70131203]
 [ 11.88151419 -20.83204062  12.12392023]]
[3.141592653589793, 0.0, 3.141592653589793]
E_0 = -3.141592653589793
1
[[ 1.      0.8378  0.3928 -0.1886]
 [ 0.8378  0.3928 -0.1886 -0.6992]
 [ 0.3928 -0.1886 -0.6992 -0.9782]]
X

-2.0